# RNN and Transformers


In this lab lesson we will see how and when to use Recurrent Neural Networks and how to exploits pre-trained Transformers model like Bert.


## Task description



In this exercise we will try to classify subjectivity of text in sentences.

We will use a collection of 103 Italian newspaper's articles labeled as Objective or Subjective. Each article is divided in sentences, which are consequently classified as either Subjective or Objective.

You can find the data along with a more detailed description [here](https://github.com/francescoantici/SubjectivITA).

We will be trying to create a model which is able to predict if a sentence contains subjectivity or it is fully objective.



## Import libraries

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

## Data loading

Please download the train, val and test files from the [repository](https://github.com/francescoantici/SubjectivITA/tree/main/datasets/sentences).

After having uploaded the three files to the notebook, use this utility function to load the data.

In [2]:
def get_data_split(split):
  """
  Args:
    - split: the split of the data you want to load.
  Returns:
    - X, y data, where X is the array containing the sentences and y is the labels vector.

  """
  df = pd.read_csv(f"sentences{split.capitalize()}.csv")
  return df['FRASE'].values, df['TAG_FRASE'].values

In [3]:
sentences_train, labels_train = get_data_split(split = 'train')
sentences_val, labels_val = get_data_split(split = 'val')
sentences_test, labels_test = get_data_split(split = 'test')

### Data Inspection

In [4]:
sentences_train

array(['Prova estrema su TikTok:',
       'bambina di 10 anni in coma a Palermo, dichiarata la morte cerebrale.',
       'Inchiesta per istigazione al suicidio.', ...,
       'è quanto ha detto Guido Bertolaso, nuovo consulente della Lombardia per la campagna vaccinale regionale, nel corso di una conferenza stampa con il presidente Attilio Fontana e la vicepresidente Letizia Moratti.',
       'Non voglio soldi, faccio il volontario e mi sono abbassato lo stipendio: da un euro zero,',
       'ha aggiunto.'], dtype=object)

In [5]:
labels_train

array(['OGG', 'OGG', 'OGG', ..., 'OGG', 'SOG', 'OGG'], dtype=object)

## RNN

Recurrent neural networks (RNN) are a class of neural networks that is powerful for
modeling sequence data such as time series or natural language.

Schematically, a RNN layer uses a `for` loop to iterate over the timesteps of a
sequence, while maintaining an internal state that encodes information about the
timesteps it has seen so far.

The Keras RNN API is designed with a focus on:

- **Ease of use**: the built-in `keras.layers.RNN`, `keras.layers.LSTM`,
`keras.layers.GRU` layers enable you to quickly build recurrent models without
having to make difficult configuration choices.

- **Ease of customization**: You can also define your own RNN cell layer (the inner
part of the `for` loop) with custom behavior, and use it with the generic
`keras.layers.RNN` layer (the `for` loop itself). This allows you to quickly
prototype different research ideas in a flexible way with minimal code.

There are three built-in RNN layers in Keras:

1. `keras.layers.SimpleRNN`, a fully-connected RNN where the output from previous
timestep is to be fed to next timestep.

2. `keras.layers.GRU`, first proposed in
[Cho et al., 2014](https://arxiv.org/abs/1406.1078).

3. `keras.layers.LSTM`, first proposed in
[Hochreiter & Schmidhuber, 1997](https://www.bioinf.jku.at/publications/older/2604.pdf).

### Data pre-processsing

We will use a tokenizer [function](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer) provided by Keras to map each token to an integer, so that the model is able to interpreter it.

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer


lbl_to_idx_dict = {"OGG":0, "SOG":1}

label_to_idx_f = np.vectorize(lbl_to_idx_dict.get)

vocabulary_dim = 10000

def get_tokenizer(x_train): #all the words are encoded in a dictionary as numbers. kearas has a tokenizer method which does it for us.
  tokenizer = Tokenizer(num_words = vocabulary_dim)
  tokenizer.fit_on_texts(x_train)
  return tokenizer

tokenizer = get_tokenizer(sentences_train)

In [11]:
tokenizer.word_index

{'di': 1,
 'e': 2,
 'il': 3,
 'che': 4,
 'la': 5,
 'a': 6,
 'in': 7,
 'per': 8,
 'è': 9,
 'un': 10,
 'i': 11,
 'del': 12,
 'della': 13,
 'non': 14,
 'ha': 15,
 'le': 16,
 'con': 17,
 'si': 18,
 'una': 19,
 'sono': 20,
 'da': 21,
 'al': 22,
 'più': 23,
 'dei': 24,
 'alla': 25,
 'anche': 26,
 'nel': 27,
 'ma': 28,
 'delle': 29,
 'hanno': 30,
 'gli': 31,
 'come': 32,
 'lo': 33,
 'sul': 34,
 'nella': 35,
 'su': 36,
 'stato': 37,
 'se': 38,
 'anni': 39,
 'due': 40,
 'stati': 41,
 'dal': 42,
 'dopo': 43,
 'fatto': 44,
 'loro': 45,
 'governo': 46,
 'tra': 47,
 'prima': 48,
 'dalla': 49,
 'presidente': 50,
 'alle': 51,
 'ai': 52,
 'era': 53,
 'essere': 54,
 'parte': 55,
 'degli': 56,
 'nei': 57,
 'poi': 58,
 'o': 59,
 'tutti': 60,
 'giorni': 61,
 'questo': 62,
 'cui': 63,
 'solo': 64,
 'ancora': 65,
 'aveva': 66,
 'sulla': 67,
 'ci': 68,
 'via': 69,
 'nelle': 70,
 'questa': 71,
 'casa': 72,
 'persone': 73,
 'altri': 74,
 'giorno': 75,
 'quando': 76,
 'nuovo': 77,
 'suo': 78,
 'sempre': 79,
 't

In [12]:
from tensorflow.keras.utils import pad_sequences

maxSentenceLen = 20 #taken after analysis -> sentences are almost never longer than 20. we truncate longer sentences with keras methods

generate_x = lambda x: pad_sequences(tokenizer.texts_to_sequences(x), maxlen = maxSentenceLen, padding = "post")

x_train = generate_x(sentences_train)
x_test = generate_x(sentences_test)
x_val = generate_x(sentences_val)

y_train = label_to_idx_f(labels_train)
y_test = label_to_idx_f(labels_test)
y_val = label_to_idx_f(labels_val)

In [13]:
x_train.shape #1399 sentences with 20 words each (0 = empty word to arrive at 20 if it's shorter)

(1399, 20)

In [15]:
x_train

array([[ 492, 1090,   36, ...,    0,    0,    0],
       [ 127,    1,  419, ...,    0,    0,    0],
       [2715,    8, 1091, ...,    0,    0,    0],
       ...,
       [   5,  165,  431, ..., 1025, 7126, 7127],
       [  14, 1379, 2601, ...,    0,    0,    0],
       [  15,  652,    0, ...,    0,    0,    0]], dtype=int32)

### Models

let's build a rnn baseline based on LSTM

In [16]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

#notice there are no activations
def get_rnn_model(input_shape, out_dim, vocabulary_dim):
  input = Input(shape=input_shape)

  embedding_layer = Embedding(input_dim=vocabulary_dim, output_dim=64)(input) #embeding word tokens into vector space (can also be one-hot encoding)
  #layer that has a grasp of contest
  lstm_1 = LSTM(128, return_sequences=True, recurrent_dropout = 0.2)(embedding_layer) #use return_sequence, to output for each sequence and not just the last

  lstm_2 = LSTM(64, dropout = 0.2)(lstm_1) #don't return sequences

  output = Dense(out_dim)(lstm_2)

  model = Model(input, output)

  model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer = Adam(1e-3), metrics = ['accuracy'])
  
  model.summary()
  
  return model

In [17]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', mode = 'max', patience = 5, restore_best_weights = True) #callback stops training automatically when detecting overfitting

In [18]:
model = get_rnn_model((20,), 2, vocabulary_dim)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20)]              0         
                                                                 
 embedding (Embedding)       (None, 20, 64)            640000    
                                                                 
 lstm (LSTM)                 (None, 20, 128)           98816     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 788,354
Trainable params: 788,354
Non-trainable params: 0
_________________________________________________________________


In [19]:
history = model.fit(x_train, y_train, epochs=10, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/10
44/44 [==============================] - 26s 365ms/step - loss: 0.6129 - accuracy: 0.7091 - val_loss: 0.6595 - val_accuracy: 0.6233
Epoch 2/10
44/44 [==============================] - 14s 313ms/step - loss: 0.4257 - accuracy: 0.8056 - val_loss: 0.6893 - val_accuracy: 0.7442
Epoch 3/10
44/44 [==============================] - 9s 213ms/step - loss: 0.1056 - accuracy: 0.9593 - val_loss: 0.9056 - val_accuracy: 0.7209
Epoch 4/10
44/44 [==============================] - 5s 125ms/step - loss: 0.0591 - accuracy: 0.9836 - val_loss: 1.3830 - val_accuracy: 0.6698
Epoch 5/10
44/44 [==============================] - 6s 135ms/step - loss: 0.0176 - accuracy: 0.9950 - val_loss: 1.7175 - val_accuracy: 0.6837
Epoch 6/10
44/44 [==============================] - 4s 93ms/step - loss: 0.0093 - accuracy: 0.9979 - val_loss: 1.5534 - val_accuracy: 0.6605
Epoch 7/10
44/44 [==============================] - 5s 118ms/step - loss: 0.0047 - accuracy: 0.9986 - val_loss: 1.7907 - val_accuracy: 0.6884


### Model evaluation

For the model evaluation we will use the [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) function provided by scikit-learn, which will present a detailed report of the performances of the model evaluated on different metrics.

In [20]:
toLabels = np.vectorize(lambda e: "OGG" if e == 0 else "SOG")

def evaluate_model(model, x_test, y_test):
  """
  Args:
    - model: the model to use to make the prediction.
    - x_test: the sentences to label.
    - y_test: the actual labels.
  Returns:
    - The results of the evaluation of the model.

  """
  y_pred = np.argmax(model.predict(x_test), axis = -1)
  #classification report gives a confusion matrix if we're doing classification
  print(classification_report(toLabels(y_test), y_pred = toLabels(y_pred)))

In [21]:
evaluate_model(model, x_test, y_test)

8/8 [==============================] - 1s 16ms/step
              precision    recall  f1-score   support

         OGG       0.79      0.76      0.78       152
         SOG       0.56      0.60      0.58        75

    accuracy                           0.71       227
   macro avg       0.68      0.68      0.68       227
weighted avg       0.72      0.71      0.71       227



#### Bidirectional RNNs

For sequences other than time series (e.g. text), it is often the case that a RNN model
can perform better if it not only processes sequence from start to end, but also
backwards. For example, to predict the next word in a sentence, it is often useful to
have the context around the word, not only just the words that come before it.

Keras provides an easy API for you to build such **bidirectional RNNs**: the
`keras.layers.Bidirectional` wrapper.

In [23]:
from tensorflow.keras.layers import Bidirectional, TimeDistributed
#it's always worth a try to add a bidirectional layer when dealing with sequences, to see if it improves our predictions (might better understand contex)
def get_rnn_model_bd(input_shape, out_dim, vocabulary_length):
  input = Input(shape=input_shape)

  embedding_layer = Embedding(input_dim=vocabulary_length, output_dim=64)(input)
  #same as before, but using bidiretional instead of vanilla LSTM
  lstm_1 = Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout = 0.2))(embedding_layer) 

  lstm_2 = Bidirectional(LSTM(64, dropout = 0.2))(lstm_1)

  output = Dense(out_dim)(lstm_2)

  model = Model(input, output)

  model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer = Adam(1e-3), metrics = ['accuracy'])
  
  model.summary()
  
  return model

In [24]:
model_bd = get_rnn_model_bd((20,), 2, vocabulary_dim)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 20, 64)            640000    
                                                                 
 bidirectional (Bidirectiona  (None, 20, 256)          197632    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1,002,242
Trainable params: 1,002,242
Non-tra

In [25]:
history = model_bd.fit(x_train, y_train, epochs=10, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/10
44/44 [==============================] - 23s 341ms/step - loss: 0.6189 - accuracy: 0.7134 - val_loss: 0.6630 - val_accuracy: 0.6233
Epoch 2/10
44/44 [==============================] - 10s 222ms/step - loss: 0.4727 - accuracy: 0.7655 - val_loss: 0.8251 - val_accuracy: 0.7442
Epoch 3/10
44/44 [==============================] - 10s 234ms/step - loss: 0.1547 - accuracy: 0.9414 - val_loss: 0.7473 - val_accuracy: 0.7023
Epoch 4/10
44/44 [==============================] - 10s 226ms/step - loss: 0.0412 - accuracy: 0.9878 - val_loss: 1.0008 - val_accuracy: 0.6930
Epoch 5/10
44/44 [==============================] - 10s 219ms/step - loss: 0.0133 - accuracy: 0.9971 - val_loss: 1.8902 - val_accuracy: 0.6977
Epoch 6/10
44/44 [==============================] - 8s 186ms/step - loss: 0.0173 - accuracy: 0.9936 - val_loss: 1.8653 - val_accuracy: 0.6744
Epoch 7/10
44/44 [==============================] - 9s 202ms/step - loss: 0.0066 - accuracy: 0.9979 - val_loss: 1.8972 - val_accuracy: 0.6837


In [26]:
evaluate_model(model_bd, x_test, y_test)

8/8 [==============================] - 1s 15ms/step
              precision    recall  f1-score   support

         OGG       0.80      0.76      0.78       152
         SOG       0.56      0.61      0.59        75

    accuracy                           0.71       227
   macro avg       0.68      0.69      0.68       227
weighted avg       0.72      0.71      0.72       227



## Transformers

Transformers are deep neural networks that over the last years achieved state of the art performances in several tasks.

Transformers replaces CNNs and RNNs with [self-attention](https://developers.google.com/machine-learning/glossary#self-attention). Self attention allows Transformers to easily transmit information across the input sequences.

For the transformer model implementation we will rely on a Python library called `transformers`, which provides an API inteface to several pre-trained models for fine-tuning or transfer-learning purposes.

In [27]:
!pip3 install transformers 
from transformers import AutoTokenizer, TFBertModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00


### Pre-trained model

For this task we will use a pre-trained language model called [AlBERTo](github.com/marcopoli/AlBERTo-it). AlBERTo is a BERT model trained for the Italian language. In particular, AlBERTo is focused on the language used in social networks, specifically on Twitter. Due to the language and the type of data present in the dataset AlBERTo is the best fit for this kind of task.

You can find the pre-trained model in [huggingface](https://huggingface.co/bert-base-multilingual-cased?text=mi+piace+il+%5BMASK%5D), which is an open repository for pre-trained architectures, available in both pytorch and tensorflow (depending on the developers).

### Data pre-processing

Transoformes must recieve input in a standard format, namely divided in `input_ids`, `token_type_ids`, `attention_mask`.

In [ ]:
def prepare_data_bert(x, y, maxSentenceLen = maxSentenceLen):
  """
  Args:
    - x: the sentences to label.
    - y: the actual labels.
    - maxSentenceLen: The maximum length of the sentences, it is used as a truncation length
  Returns:
    - A tuple with the input to feed into a transformers model, namely ((input_ids, attention_mask, token_type_ids), categorical_labels).

  """
  pad = tf.keras.preprocessing.sequence.pad_sequences
  tokenizer = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")
  dataFields = {
          "input_ids": [],
          "token_type_ids": [],
          "attention_mask": [],
          "subjectivity": []
      }
  lbls = {
      'SOG' : 1.0,
      'OGG' : 0.0
  }
  for i in range(len(x)):
      data = tokenizer(x[i])
      padded = pad([data['input_ids'], data['attention_mask'], data['token_type_ids']], padding = 'post', maxlen = maxSentenceLen)
      dataFields['input_ids'].append(padded[0])
      dataFields['attention_mask'].append(padded[1])
      dataFields['token_type_ids'].append(padded[-1])
      dataFields['subjectivity'].append(lbls[y[i]])
  
  for key in dataFields:
      dataFields[key] = np.array(dataFields[key])
  #returns 3 inputs (based on attention), 1 output vector
  return [dataFields["input_ids"], dataFields["token_type_ids"], dataFields["attention_mask"]], dataFields["subjectivity"] 

### Model

model has to take a list of 3 inputs, a bert layer and a dense output.

In [ ]:
def create_transformers_model(input_shape, out_dim):


In [ ]:
bert_model= create_transformers_model(_,_)

In [ ]:
def create_transformers_model(input_shape, out_dim):

    input_ids = Input(shape=input_shape, dtype=tf.int32)
    token_type_ids = Input(shape=input_shape, dtype=tf.int32)
    attention_mask = Input(shape=input_shape, dtype=tf.int32)

    bertModel = TFBertModel.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[-1]

    out = Dense(out_dim, activation=tf.nn.sigmoid)(tf.keras.layers.Dropout(0.1)(bertModel))

    model = Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=out)

    model.compile(optimizer = Adam(1e-5), loss = SparseCategoricalCrossentropy(from_logits=True), metrics = ['accuracy'])

    return model

In [ ]:
bert_model = create_transformers_model((20,), 2)

In [ ]:
x_train_bert, y_train_bert = prepare_data_bert(sentences_train, labels_train)
x_val_bert, y_val_bert = prepare_data_bert(sentences_val, labels_val)

In [ ]:
history = bert_model.fit(x_train_bert, 
                         y_train_bert, 
                         epochs=4, 
                         validation_data = (x_val_bert, y_val_bert), 
                         batch_size = 16, 
                         callbacks = [callback]
                         )

### Model evaluation

In [ ]:
x_test_bert, y_test_bert = prepare_data_bert(sentences_test, labels_test)

evaluate_model(bert_model, x_test_bert, y_test_bert)